# Singular Value Decomposition of the snapshots

At first, let us define the mesh onto which the snapshots are defined

In [1]:
from mesh import *

domain, ct, ft = evol_mesh()

Info    : Reading '../EVOL_geom.geo'...
Info    : Done reading '../EVOL_geom.geo'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 40%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 70%] Meshing curve 10 (Line)
Info    : [ 80%] Meshing curve 11 (Line)
Info    : [ 90%] Meshing curve 12 (Line)
Info    : [100%] Meshing curve 13 (Line)
Info    : Done meshing 1D (Wall 0.00166207s, CPU 0.001213s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 1.37676s, CPU 1.35797s)
Info    : 45732 nodes 92055 elements
Info    : Optimizing mesh (Netgen)...
Info    : Done optimizing mesh (W

Let us define the functional spaces and let us import the data

In [2]:
from dolfinx.fem import FunctionSpace
import ufl
from pyforce.tools.functions_list import FunctionsList
from pyforce.tools.write_read import ImportH5
import pickle

var_names, is_vector, times = pickle.load(open('msfr_uloff.set', 'rb'))

fun_spaces = [FunctionSpace(domain, ('Lagrange', 1))]*(len(var_names)-1)
fun_spaces.append(FunctionSpace(domain, ufl.VectorElement("CG", domain.ufl_cell(), 1)))

snaps = dict()
path_snaps = './dolfinx_data/'

for field_i, field in enumerate(var_names):
    print('Importing '+field)
    snaps[field] = ImportH5(fun_spaces[field_i], path_snaps+field, field, verbose=False)[0]

Importing flux1
Importing flux2
Importing flux3
Importing flux4
Importing flux5
Importing flux6
Importing prec1
Importing prec2
Importing prec3
Importing prec4
Importing prec5
Importing prec6
Importing prec7
Importing prec8
Importing dec1
Importing dec2
Importing dec3
Importing T
Importing p
Importing U


In [3]:
var_names

['flux1',
 'flux2',
 'flux3',
 'flux4',
 'flux5',
 'flux6',
 'prec1',
 'prec2',
 'prec3',
 'prec4',
 'prec5',
 'prec6',
 'prec7',
 'prec8',
 'dec1',
 'dec2',
 'dec3',
 'T',
 'p',
 'U']

Let us define a function to return a matrix from `FunctionsList`

In [4]:
import numpy as np

def return_matrix(_snaps: FunctionsList):
    matrix = np.zeros((_snaps.fun_shape[0], len(_snaps)))
    for mu in range(len(_snaps)):
        matrix[:, mu] = _snaps(mu)
        
    return matrix

## Randomized SVD

In [5]:
from sklearn.utils.extmath import randomized_svd
from pyforce.tools.functions_list import FunctionsMatrix

rescaling_snaps = list()
new_var_names = list()

Nmodes = 10

u_total = dict()
s_total = dict()

for field_i, field in enumerate(var_names):
    
    print(field)
    u = return_matrix(snaps[field])
    rescaling_snaps.append(np.mean(u[:,0]))

    X = u / rescaling_snaps[field_i]
    # print(min(X.flatten()))
    # print(max(X.flatten()))
    
    u_svd, s_svd, v_svd = randomized_svd(X, n_components=Nmodes,n_iter='auto')

    if field_i == 0:
        v_total = v_svd
    else:
        v_total = np.vstack((v_total, v_svd))
        
    s_total[field] = s_svd.flatten()
    u_total[field] = u_svd
        
    new_var_names.append(field)
        
# rescaling_snaps = np.asarray(rescaling_snaps)

flux1
flux2
flux3
flux4
flux5
flux6
prec1
prec2
prec3
prec4
prec5
prec6
prec7
prec8
dec1
dec2
dec3
T
p
U


In [6]:
print(v_total.shape)

(200, 500)


Let us plot the singular values

In [7]:
# import matplotlib.pyplot as plt
# from matplotlib import cm

# fig, axs = plt.subplots(nrows = 1, ncols = 2, sharex=True, figsize=(12,5))

# N_plot = np.arange(1, Nmodes+1, 1)

# colors = cm.jet(np.linspace(0,1,len(var_names)))
# for field_i, field in enumerate(var_names):
#     axs[0].semilogy(N_plot, s_total[field], color=colors[field_i], label=field)
#     axs[1].semilogy(N_plot, 1-np.cumsum(s_total[field]**2) / np.sum(s_total[field]**2), color=colors[field_i], label=field)
# axs[0].grid()
# axs[1].grid()
# axs[0].legend()

In [8]:
from scipy.interpolate import CubicSpline
from pyforce.tools.write_read import StoreFunctionsList
import os

[r,nt] = v_total.shape
t = np.asarray(times)
newnt = 2000

new_t = np.linspace(t[0], t[-1], newnt)
v_total_new = np.zeros((r,newnt))

for j in range(r):
    spline = CubicSpline(t, v_total[j,:])
    v_total_new[j,:] = spline(new_t)


path='./OfflineSVD/'
if not os.path.exists(path):
    os.makedirs(path)
    
pickle.dump(u_total, open(path+"u_total.svd", 'wb'))
pickle.dump(v_total_new, open(path+"v_total.svd", 'wb'))
pickle.dump(s_total, open(path+"s_total.svd", 'wb'))
pickle.dump(new_var_names, open(path+"var.names", 'wb'))

pickle.dump(rescaling_snaps, open(path+"rescaling.svd", 'wb'))

In [9]:
# import matplotlib.pyplot as plt

# plt.plot(new_t, v_total_new[30:35].T)
# plt.plot(times, v_total[30:35].T, '--')